# Giới thiệu Convolution Nets

Convolutional Neural Networks (CNN) là một trong những mô hình deep learning phổ biến nhất và có ảnh hưởng nhiều nhất trong cộng đồng Computer Vision. CNN được dùng trong trong nhiều bài toán như nhận dạng ảnh, phân tích video, ảnh MRI, hoặc cho bài các bài của lĩnh vực xử lý ngôn ngữ tự nhiên, và hầu hết đều giải quyết tốt các bài toán này. 

CNN cũng có lịch sử khá lâu đời. Kiến trúc gốc của mô hình CNN được giới thiệu bởi một nhà khoa học máy tính người Nhật vào năm 1980. Sau đó, năm 1998, Yan LeCun lần đầu huấn luyện mô hình CNN với thuật toán backpropagation cho bài toán nhận dạng chữ viết tay. Tuy nhiên, mãi đến năm 2012, khi một nhà khoa học máy tính người Ukraine Alex Krizhevsky (đệ của Geoffrey Hinton) xây dựng mô hình CNN (AlexNet) và sử dụng GPU để tăng tốc quá trình huấn luyện deep nets để đạt được top 1 trong cuộc thi Computer Vision thường niên ImageNet với độ lỗi phân lớp top 5 giảm hơn 10% so với những mô hình truyền thống trước đó, đã tạo nên làn sóng mãnh mẽ sử dụng deep CNN với sự hỗ trợ của GPU để giải quyết càng nhiều các vấn đề trong Computer Vision.

# Bài Toán Phân loại Ảnh
Phân loại ảnh là một bài toán quan trọng bậc nhất trong lĩnh vực Computer Vision. Chúng ta đã có rất nhiều nghiên cứu để giải quyết bài toán này bằng cách rút trích các đặc trưng rất phổ biến như SIFT, HOG rồi cho máy tính học nhưng những cách này tỏ ra không thực sự hiểu quả. Nhưng ngược lại, đối với con người, chúng ta lại có bản năng tuyệt vời để phân loại được những đối tượng trong khung cảnh xung quanh một cách dễ dàng.

Dữ liệu đầu vào của bài toán là một bức ảnh. Một ảnh được biểu diễn bằng ma trận các giá trị. Mô hình phân lớp sẽ phải dự đoán được lớp của ảnh từ ma trận điểm ảnh này, ví dụ như ảnh đó là con mèo, chó, hay là chim.

![](https://pbcquoc.github.io/images/cnn_input.png)

# Nội dung 
Trong assignment này, mình sẽ hướng dẫn các bạn xây dựng mô hình CNN (Convolution Neural Nets) cho bài toán phân loại ảnh. Các bạn sẽ sử dụng tensorflow để xây dựng model, huấn luyện mô hình trên tập train và predict ảnh trong tập test. 

Assignment này sẽ có câú trúc như sau:
1. Import/ Xử lý dữ liệu
2. Xây dựng mô hình
3. Huấn luyện mô hình
4. Đánh giá mô hình
5. Sử dụng mô hình đã huấn luyện để dự đoán

# Import thư viện

Chúng ta sử dụng một số hàm cơ bản trong tensorflow, sklearn và phải gọi hàm tf.enable_eager_execution. 

In [ ]:
import os
import numpy as np
np.warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
from tqdm import tqdm

from google_drive_downloader import GoogleDriveDownloader as gdd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from PIL import Image

tf.random.set_seed(0)
np.random.seed(0)

print("Tensorflow version: ", tf.__version__)

Tensorflow version:  2.4.1


# Import và inspect dữ liệu
Trong bài này, các bạn phải xây dựng mô hình để phân loại 10 địa danh của Việt Nam. Tập dữ liệu huấn luyện bao gồm hơn 8 ngàn ảnh. 


## Download dữ liệu
Bạn có thể sử dụng trực tiếp dữ liệu trên competition được host trên Kaggle: [VietAI Foundation Course - CNN Assignment](https://www.kaggle.com/c/vietai-foundation-course-7-cnn-assignment/data)

Hoặc tải dữ liệu xuống từ Google Drive

In [ ]:
gdd.download_file_from_google_drive(file_id='1wjuPDgCDThA2vk-2ctpTxr1HMoYYnm7h', dest_path='./Assignment3.zip', unzip=True)

Dữ liệu tải xuống sẽ chứa trong folder `data`. Cấu trúc thư mục như sau:

In [ ]:
data_dir = 'Assignment3'
os.listdir(data_dir)

['train.csv', 'sample_submission.csv', 'images']

Trong đó:
- **images**: thư mục chứa tất cả các ảnh dùng cho việc huấn luyện và đánh giá
- **train.csv**: file CSV chứa tên các file và nhãn dùng cho việc huấn luyện
- **sample_submission.csv**: file CSV mẫu chứa tên các file cần đánh giá và nhãn dummy.

## Đọc và xử lý dữ liệu

Đọc dữ liệu từ file CSV:

In [ ]:
train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
train_df.head()

,image,label
0,VietAI-Assignment3-1.jpg,7
1,VietAI-Assignment3-100.jpg,2
2,VietAI-Assignment3-10000.jpg,1
3,VietAI-Assignment3-10001.jpg,2
4,VietAI-Assignment3-10002.jpg,2


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8234 entries, 0 to 8233
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   8234 non-null   object
 1   label   8234 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 128.8+ KB


In [ ]:
test_df = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
test_df.head()

,image,label
0,VietAI-Assignment3-10.jpg,0
1,VietAI-Assignment3-1000.jpg,0
2,VietAI-Assignment3-10004.jpg,0
3,VietAI-Assignment3-10006.jpg,0
4,VietAI-Assignment3-10012.jpg,0


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   2059 non-null   object
 1   label   2059 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 32.3+ KB


Tổng cộng có 8,324 ảnh cho việc huấn luyện và 2,059 ảnh cần dự đoán nhãn, ta tiến hành thống kê phân bố các nhãn trên tập huấn luyện:

In [ ]:
train_df.label.value_counts()

2     1949
1      846
0      808
3      764
5      642
4      599
7      579
10     575
6      535
8      469
9      468
Name: label, dtype: int64

Số lượng các ảnh cho mỗi lớp từ xấp xỉ 470 đến xấp xỉ 2000. Trong đó lớp số 2 có số lượng ảnh nhiều nhất.

Đoạn code bên dưới show ra 1 số hình ảnh và class của nó.

In [ ]:
import cv2
def show_gallery(df, n=5, shuffle=True):
  plt.subplots(figsize=(20,20))
  if shuffle:
    df = df.sample(frac=1).reset_index(drop=True)
  k=1
  for i in range(n*n):
    im = cv2.imread(os.path.join("./Assignment3/images",df.loc[k,"image"]))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    label = df.loc[k,"label"]
    plt.subplot(n,n,k)
    plt.imshow(im)
    plt.title("Label: {}".format(label))
    plt.axis("off")
    k += 1

show_gallery(train_df)

Output hidden; open in https://colab.research.google.com to view.

## TODO 1: Cài đặt hàm đọc ảnh và đưa về NumPy Array
Để máy tính hiểu được các ảnh, chúng ta cần đọc và chuyển các ảnh về tensor. Bên cạnh đó, các tensor biểu diễn cần có kích thước cố định nên trong quá trình đọc ảnh, ta cần thay đổi về kích thước mong muốn (resize ảnh). Trong các bài toán về deep learning, ta thường biểu diễn ảnh dưới dạng tensor có kích thước `(224,224,3)` với 3 kênh màu, 224 pixels cho mỗi kênh.

Hoàn thành hàm `generate_data` bên dưới nhận vào 1 list N đường dẫn đến ảnh và kích thước `size` ảnh cần resize. Trả về numpy array có kích thước `(N,size,size,3)`. Do ảnh màu có thể biểu diễn bằng số nguyên 0 --> 255 nên để tiết kiệm bộ nhớ cho phù hợp với ram colab, chỉ cần dùng kiểu dữ liệu uint8.

In [ ]:
def generate_data(image_paths, size=224):
    """
    Đọc và chuyển các ảnh về numpy array
    
    Parameters
    ----------
    image_paths: list of N strings
        List các đường dẫn ảnh
    size: int
        Kích thước ảnh cần resize
    
    Returns
    -------
    numpy array kích thước (N, size, size, 3)
    """
    image_array = np.zeros((len(image_paths), size, size, 3), dtype='uint8')
    
    for idx, image_path in tqdm(enumerate(image_paths)):
        ### START CODE HERE
        
        # Đọc ảnh bằng thư viện Pillow và resize ảnh
        image = Image.open(image_path)
        image = image.resize((224,224))
        
        # Chuyển ảnh thành numpy array và gán lại mảng image_array
        image_array[idx] = np.asarray(image, dtype='uint8')
        
        ### END CODE HERE
    return image_array

Sử dụng hàm `generate_data` để tạo ma trận của tập dữ liệu train và test:

In [ ]:
# List các đường dẫn file cho việc huấn luyện
train_files = [os.path.join("Assignment3/images", file) for file in train_df.image]

# List các nhãn
train_y = train_df.label

# Tạo numpy array cho dữ liệu huấn luyện
train_arr = generate_data(train_files)

8234it [00:56, 146.32it/s]


Hãy kiểm tra kích thước của tensor `train_arr` vừa tạo ra. Kích thước đúng sẽ là `(8234,224,224,3)`.

In [ ]:
train_arr.shape

(8234, 224, 224, 3)

Tiến hành tạo tensor dữ liệu cho tập test:

In [ ]:
test_files = [os.path.join("Assignment3/images", file) for file in test_df.image]
test_x = generate_data(test_files)
test_x.shape

2059it [00:12, 159.83it/s]


(2059, 224, 224, 3)

Tạo **one-hot labels** từ `train_y` để đưa vào huấn luyện với Tensorflow. 

In [ ]:
num_classes = len(np.unique(train_y))
y_ohe = tf.keras.utils.to_categorical(train_y, num_classes=num_classes)

## Chia dữ liệu để huấn luyện và đánh giá

Ta sẽ không sử dụng 100% tập dữ liệu đã có nhãn để huấn luyện mà sẽ chỉ huấn luyện trên 75% bộ dữ liệu và sử dụng 25% còn lại dùng để đánh giá model qua các epoch.

Chúng ta sử dụng hàm `train_test_split` trong thư viện sklearn để chia tập dữ liệu thành 2 phần train/validation một cách nhanh chóng.

In [ ]:
x_train, x_valid, y_train_ohe, y_valid_ohe = train_test_split(train_arr, y_ohe, test_size=0.25)

print("Train size: {} - Validation size: {}".format(x_train.shape, x_valid.shape))

Train size: (6175, 224, 224, 3) - Validation size: (2059, 224, 224, 3)


## Mô Hình CNN

CNN bao gồm tập hợp các lớp cơ bản sau: convolutional layer + nonlinear layer (RELU, ...), pooling layer, fully connected layer. Các lớp này liên kết với nhau theo một thứ tự nhất định. Thông thường, một ảnh sẽ được lan truyền qua tầng convolutional layer + nonlinear layer đầu tiên, sau đó các giá trị tính toán được sẽ lan truyền qua pooling layer, bộ ba convolutional layer + nonlinear layer + pooling layer có thể được lặp lại nhiều lần trong network. Và sau đó được lan truyền qua tầng fully connected layer và softmax để tính xác suất ảnh đó thuộc lớp nào.

![](https://pbcquoc.github.io/images/cnn_model.png)

### Convolutional Layer
Convolutional layer thường là lớp đầu tiên và cũng là lớp quan trọng nhất của mô hình CNN. Lớp này có chức năng chính là phát hiện các đặc trưng về không gian một cách hiệu quả. Trong tầng này có 4 đối tượng chính là: ma trận đầu vào, bộ **filter**, và **receptive field**, **feature map**. Conv layer nhận đầu vào là một ma trận 3 chiều và một bộ filter cần phải học. Bộ filters này sẽ trượt qua từng vị trí trên bức ảnh để tính tích chập (convolution) giữa bộ filter và phần tương ứng trên bức ảnh. Phần tương ứng này trên bức ảnh gọi là receptive field, tức là vùng mà một neuron có thể nhìn thấy để đưa ra quyết định, và ma trận sinh ra bởi quá trình này được gọi là feature map. Để hình dung, các bạn có thể tưởng tượng, bộ filters giống như các tháp canh trong nhà tù quét lần lượt qua không gian xung quanh để tìm kiếm tên tù nhân bỏ trốn. Khi phát hiện tên tù nhân bỏ trốn, thì chuông báo động sẽ reo lên, giống như các bộ filters tìm kiếm được đặc trưng nhất định thì tích chập đó sẽ cho giá trị lớn. 

<div class="img-div" markdown="0">
    <img src="https://media.giphy.com/media/3orif7it9f4phjv4LS/giphy.gif" />
</div>

Với ví dụ ở bên dưới, dữ liệu đầu vào là ma trận có kích thước 8x8x1, một bộ filter có kích thước 2x2x1, feature map có kích thước 7x7x1. Mỗi giá trị ở feature map được tính bằng tổng của tích các phần tử tương ứng của bộ filter 2x2x1 với receptive field trên ảnh. Và để tính tất cả các giá trị cho feature map, các bạn cần trượt filter từ trái sang phải, từ trên xuống dưới. Do đó, các bạn có thể thấy rằng phép convolution bảo toàn thứ tự không gian của các điểm ảnh. Ví dụ điểm góc trái của dữ liệu đầu vào sẽ tương ứng với bên một điểm bên góc trái của feature map. 

<div class="img-div" markdown="0">
    <img src="https://pbcquoc.github.io/images/cnn_covolution_layer.png" />
</div>

#### Tầng convolution như là feature detector 

Tầng convolution có chức năng chính là phát hiện đặc trưng cụ thể của bức ảnh. Những đặc trưng này bao gồm đặc trưng cơ bản là góc, cạnh, màu sắc, hoặc đặc trưng phức tạp hơn như texture của ảnh. Vì bộ filter quét qua toàn bộ bức ảnh, nên những đặc trưng này có thể nằm ở vị trí bất kì trong bức ảnh, cho dù ảnh bị xoay trái/phải thì những đặc trưng này vẫn được phát hiện. 

Ở minh họa dưới, các bạn có một filter 5x5 dùng để phát hiện góc/cạnh, filter này chỉ có giá trị một tại các điểm tương ứng một góc cong. 

<div class="img-div" markdown="0">
    <img src="https://pbcquoc.github.io/images/cnn_high_level_feature.png" />
</div>

Dùng filter ở trên trượt qua ảnh của nhân vật Olaf trong trong bộ phim Frozen. Chúng ta thấy rằng, chỉ ở những vị trí trên bức ảnh có dạng góc như đặc trưng ở filter thì mới có giá trị lớn trên feature map, những vị trí còn lại sẽ cho giá trị thấp hơn. Điều này có nghĩa là, filter đã phát hiện thành công một dạng góc/cạnh trên dữ liệu đầu vào. Tập hợp nhiều bộ filters sẽ cho phép các bạn phát hiện được nhiều loại đặc trưng khác nhau, và giúp định danh được đối tượng. 

<div class="img-div" markdown="0">
    <img src="https://pbcquoc.github.io/images/cnn_high_level_feature_ex.png" />
</div>

#### Các tham số của tầng convolution: Kích thước bộ filters, stride và padding

Kích thước bộ filters là một trong những siêu tham số quan trọng nhất của tầng convolution. Kích thước này tỉ lệ thuận với số lượng tham số cần học tại mỗi tầng convolution và là tham số quyết định receptive field của tầng này. Kích thước phổ biến nhất của bộ filter là 3x3.


# Xây dựng mô hình
Các bạn cần phải xây dựng mô hình CNN có kiến trúc sau đây. Bộ filter có kích thước 3x3. Đối với các tham số còn lại, các bạn có thể tự do lựa chọn để cho ra kết quả huấn luyện tốt nhất.

![](https://github.com/pbcquoc/cnn/raw/master/images/cnn_architecture_2.png)


## Định nghĩa block CNN
Để hỗ trợ quá trình định nghĩa mô hình. Các bạn cần định nghĩa một block bao gồm 3 lớp sau: Conv2D, MaxPool2D, ReLU. Block này sẽ được tái sử dụng nhiều lần trong networks. Các layers cần được khai báo trong hàm init và được gọi trong hàm call. Hãy tham khảo ví dụ dưới đây.

```python

class ConvBlock(tf.keras.Model):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.cnn = tf.keras.layers.Conv2D(32, (3, 3), strides=(1, 1),  padding="same")
        
    def call(self, inputs, training=None, mask=None):
        x = self.cnn(inputs)

        return x
```

Các tài liệu tham khảo:
- [tf.keras.layers.Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
- [tf.keras.layers.MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)

In [ ]:
class ConvBlock(tf.keras.Model):
    def __init__(self, filters, kernel, strides, padding):
        '''
        Khởi tạo Convolution Block với các tham số đầu vào
        
        Parameters
        ----------
        filters: int
            số lượng filter
        kernel: int
            kích thước kernel
        strides: int
            stride của convolution layer
        padding: str
            Loại padding của convolution layer
        
        '''
        
        super(ConvBlock, self).__init__()
        ## TODO 2
        ### START CODE HERE
        
        # Tạo layer Conv2D
        self.cnn = tf.keras.layers.Conv2D(filters=filters, 
                                          kernel_size=kernel, 
                                          strides=strides, 
                                          padding=padding, 
                                          kernel_initializer='random_normal',
                                          bias_initializer='random_normal',
                                          kernel_regularizer=tf.keras.regularizers.l2(1e-4),
                                          bias_regularizer=tf.keras.regularizers.l2(1e-4),
                                          activity_regularizer=tf.keras.regularizers.l2(1e-4),
                                          activation='relu')
        
        # Tạo layer MaxPool2D
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        
        # Tạo các layer khác tùy ý nếu cần thiết

        ### END CODE HERE
        
        
    def call(self, inputs):
        '''
        Hàm này sẽ được gọi trong quá trình forwarding của mạng
        
        Parameters
        ----------
        inputs: tensor đầu vào
        
        Returns
        -------
        tensor
            giá trị đầu ra của mạng
        '''
        
        x = None
        ## TODO 3
        ### START CODE HERE

        # Forward inputs qua từng layer và gán vào biến x để trả về
        x = self.cnn(inputs)
        x = self.pool(x)
        ## END CODE HERE

        return x

## Định nghĩa toàn bộ mô hình CNN
Các bạn sử dụng block ở trên để định nghĩa toàn bộ mô hình CNN có kiến trúc như hình dưới. Các layer cần được khởi tạo trong hàm init, và được gọi trong hàm call.
Số lượng block có thể thay đổi tùy ý các bạn nhưng phải nhiều hơn 1 block.

Trong hàm call cần phải normalize input về khoảng [0, 1] vì input hiện tại đang trong khoảng [0, 255]

In [ ]:
class CNN(tf.keras.Model):
    def __init__(self, num_classes):
        
        super(CNN, self).__init__()
        
        ## TODO 4
        ### START CODE HERE
        
        # Khởi tạo các convolution block
        self.block1 = ConvBlock(32,(3,3),(1,1),'valid')
        self.block2 = ConvBlock(64,(3,3),(1,1),'valid')
        self.block3 = ConvBlock(128,(3,3),(1,1),'valid')
        self.block4 = ConvBlock(256,(3,3),(1,1),'valid')
        self.block5 = ConvBlock(512,(3,3),(1,1),'valid')
        
        # Khởi tạo layer để flatten feature map 
        self.flatten = tf.keras.layers.Flatten()
        
        ### END CODE HERE
        
        ## TODO 5
        ### START CODE HERE
        
        # Khởi tạo fully connected layer
        self.dense1 = tf.keras.layers.Dense(512,
                                            activation='relu',
                                            kernel_regularizer=tf.keras.regularizers.l2(1e-4),
                                            bias_regularizer=tf.keras.regularizers.l2(1e-4),
                                            kernel_initializer='random_normal',
                                            bias_initializer='random_normal')
        self.drop1 = tf.keras.layers.Dropout(0.5)
        # self.dense2 = tf.keras.layers.Dense(256,
        #                                     activation='relu',
        #                                     kernel_regularizer=tf.keras.regularizers.l2(1e-4),
        #                                     bias_regularizer=tf.keras.regularizers.l2(1e-4))
        # self.drop2 = tf.keras.layers.Dropout(0.5)
        self.dense3 = tf.keras.layers.Dense(num_classes)
        ### END CODE HERE

    def call(self, inputs):
        
        ## TODO 6
        x = None
        ### START CODE HERE

        # Normalize
        inputs = inputs / 255

        # Forward gía trị inputs qua các tầng CNN và gán vào x
        x = self.block1(inputs)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        
        ### END CODE HERE
        
        ## TODO 7
        
        ### START CODE HERE 
        
        # Forward giá trị x qua Fully connected layer
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.drop1(x)
        # x = self.dense2(x)
        # x = self.drop2(x)
        x = self.dense3(x)
        
        ### END CODE HERE
        output = tf.nn.softmax(x)

        return output

## TODO 2: Cài Đặt Block CNN trong lớp ConvBlock
Sử dụng `tf.keras.layers.Conv2D` và `tf.keras.layers.MaxPool2D` để cài đặt tầng convolution và tầng pooling

## TODO 3: Gọi các tầng trong ConvBlock của lớp ConvBlock
Hãy gọi các tầng đã cài đặt trọng lớp ConvBlock trong hàm call

## TODO 4: Khai báo ConvBlock 1,2,3,4,5 trong mô hình CNN
Gọi ConvBlock đã cài đặt ở trên

## TODO 5: Khai báo Tầng Fully Connected Layer cho mô hình CNN
Gọi `tf.keras.layers.Dense` để cài đặt tầng này

## TODO 6: Gọi các tầng Conv đã khai báo trong mô hình CNN ở trên
Gọi các tầng Conv đã cài đặt

## TODO 7: Gọi tầng Fully Connected Layer
Hãy flatten tầng phía trước và gọi tầng fully connected layer để convert về ma trận có số chiều bằng số lớp cần phân loại

# Huấn Luyện
Đoạn code này thực hiện quá trình huấn luyện mô hình CNN. Mỗi lần chạy mô hình sẽ lấy `batch_size` mẫu dữ liệu, feedforward, tính loss, và cập nhật gradient cho toàn bộ trọng số. Toàn bộ quá trình này được thực hiện trong hàm `fit()` được build sẵn trong model keras.

Sau khi huấn luyện xong, chúng ta sẽ sử dụng mô hình để phân lớp các ảnh trong tập test bằng hàm `predict()`

---

Transfer Learning with Resnet50 + data aug

In [ ]:
from keras.applications.resnet50 import ResNet50


In [ ]:
# Data Generator dùng cho training
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# Data Generator dùng cho validation và testing
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
device = '/cpu:0' if len(tf.config.experimental.list_physical_devices('GPU')) == 0 else '/gpu:0'

with tf.device(device):
    base_model = ResNet50(input_shape=(224,224,3) , 
                          include_top=False, 
                          weights='imagenet')
    flat1 = tf.keras.layers.Flatten()(base_model.layers[-1].output)

    output = tf.keras.layers.Dense(num_classes, activation='softmax')(flat1)

    model = tf.keras.models.Model(inputs=base_model.inputs, outputs=output)

    # base_model.trainable = True

    mcp = tf.keras.callbacks.ModelCheckpoint("resnet50.h5", monitor="val_accuracy",
                          save_best_only=True, save_weights_only=True)
    rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, mode='max', patience=5, min_lr=1e-8, verbose=1)

    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='categorical_crossentropy',
                      metrics=['accuracy'])

    model.fit(train_datagen.flow(x_train, y_train_ohe), batch_size=32, epochs=50,
              validation_data=test_datagen.flow(x_valid, y_valid_ohe), verbose=1, callbacks=[mcp, rlr])
  

Epoch 1/50
193/193 [==============================] - 108s 482ms/step - loss: 5.1158 - accuracy: 0.4155 - val_loss: 2.3892 - val_accuracy: 0.1000
Epoch 2/50
193/193 [==============================] - 90s 466ms/step - loss: 2.4270 - accuracy: 0.4812 - val_loss: 2.3920 - val_accuracy: 0.2385
Epoch 3/50
193/193 [==============================] - 91s 468ms/step - loss: 2.7394 - accuracy: 0.4128 - val_loss: 55.4561 - val_accuracy: 0.1583
Epoch 4/50
193/193 [==============================] - 90s 468ms/step - loss: 1.9497 - accuracy: 0.4910 - val_loss: 6.6353 - val_accuracy: 0.1243
Epoch 5/50
193/193 [==============================] - 90s 467ms/step - loss: 1.7457 - accuracy: 0.5474 - val_loss: 2.1842 - val_accuracy: 0.3536
Epoch 6/50
193/193 [==============================] - 90s 467ms/step - loss: 1.3195 - accuracy: 0.6100 - val_loss: 37.9050 - val_accuracy: 0.4298
Epoch 7/50
193/193 [==============================] - 91s 470ms/step - loss: 1.3606 - accuracy: 0.6273 - val_loss: 1.0347 - val

---
Original without transfer learning & data augmentation



In [ ]:
device = '/cpu:0' if len(tf.config.experimental.list_physical_devices('GPU')) == 0 else '/gpu:0'
batch_size = 32
epochs = 32

with tf.device(device):
    # Khởi tạo model
    model = CNN(num_classes)
    
    # Tạo callback để lưu model có accuracy trên tập validation tốt nhất
    mcp = tf.keras.callbacks.ModelCheckpoint("my_model.h5", monitor="val_accuracy",
                      save_best_only=True, save_weights_only=True)
    
    # datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # horizontal_flip=True)


    # Compile model
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Huấn luyện
    # model.fit(datagen.flow(x_train, y_train_ohe), batch_size=batch_size, epochs=epochs,
    #           validation_data=(x_valid, y_valid_ohe), verbose=1, callbacks=[mcp])
    model.fit(x_train, y_train_ohe, batch_size=batch_size, epochs=epochs,
          validation_data=(x_valid, y_valid_ohe), verbose=1, callbacks=[mcp])


Epoch 1/32
193/193 [==============================] - 41s 170ms/step - loss: 4.9925 - accuracy: 0.2152 - val_loss: 2.9598 - val_accuracy: 0.3103
Epoch 2/32
193/193 [==============================] - 31s 162ms/step - loss: 2.9118 - accuracy: 0.3282 - val_loss: 2.5337 - val_accuracy: 0.4031
Epoch 3/32
193/193 [==============================] - 31s 163ms/step - loss: 2.4918 - accuracy: 0.4252 - val_loss: 2.2115 - val_accuracy: 0.4599
Epoch 4/32
193/193 [==============================] - 31s 163ms/step - loss: 2.1724 - accuracy: 0.4995 - val_loss: 1.8653 - val_accuracy: 0.5721
Epoch 5/32
193/193 [==============================] - 31s 163ms/step - loss: 1.8229 - accuracy: 0.5799 - val_loss: 1.7477 - val_accuracy: 0.5760
Epoch 6/32
193/193 [==============================] - 31s 163ms/step - loss: 1.6618 - accuracy: 0.6134 - val_loss: 1.5594 - val_accuracy: 0.6537
Epoch 7/32
193/193 [==============================] - 31s 162ms/step - loss: 1.4201 - accuracy: 0.6927 - val_loss: 1.3952 - val_ac

# Dự Đoán các ảnh trên tập test

Chúng ta sử dụng mô hình đã được huấn luyện bên trên để dự đoán cho các ảnh trong tập test, xuất ra file CSV và submit kết quả lên Kaggle:

[Link nộp kết quả](https://www.kaggle.com/c/vietai-foundation-course-7-cnn-assignment)

## Tạo và load model đã lưu trước đó

In [ ]:
# Load best model
model = CNN(num_classes)

# Thiết lập kích thước input cho model
model.build(input_shape=(1,224,224,3))

# Load model đã lưu trước đó trong quá trình huấn luyện
model.load_weights('my_model.h5')
print("Model đã được load")
print(model.summary())

Model đã được load
Model: "cnn_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_block_10 (ConvBlock)    multiple                  896       
_________________________________________________________________
conv_block_11 (ConvBlock)    multiple                  18496     
_________________________________________________________________
conv_block_12 (ConvBlock)    multiple                  73856     
_________________________________________________________________
conv_block_13 (ConvBlock)    multiple                  295168    
_________________________________________________________________
conv_block_14 (ConvBlock)    multiple                  1180160   
_________________________________________________________________
flatten_2 (Flatten)          multiple                  0         
_________________________________________________________________
dense_4 (Dense)              multiple     

## Dự đoán nhãn của các ảnh trên tập test

Sử dụng hàm predict để dự đoán:

In [ ]:
pred = model.predict(test_x)

# pred là một ma trận xác suất của ảnh trên các lớp.
# Ta lấy lớp có xác suất cao nhất trên từng ảnh bằng hàm argmax
pred_labels = np.argmax(pred, axis=1)

Hiển thị thử kết quả của tập test:

In [ ]:
test_df['label'] = pred_labels
test_df.head(20)

,image,label
0,VietAI-Assignment3-10.jpg,1
1,VietAI-Assignment3-1000.jpg,10
2,VietAI-Assignment3-10004.jpg,10
3,VietAI-Assignment3-10006.jpg,5
4,VietAI-Assignment3-10012.jpg,2
5,VietAI-Assignment3-10029.jpg,4
6,VietAI-Assignment3-10034.jpg,2
7,VietAI-Assignment3-10037.jpg,2
8,VietAI-Assignment3-1005.jpg,10
9,VietAI-Assignment3-10052.jpg,4


Lưu kết quả thành file CSV:

In [ ]:
test_df.to_csv("submission.csv", index=False)

## Nộp kết quả lên Kaggle

1. Truy cập vào [Kaggle](https://www.kaggle.com), đăng ký/ đăng nhập tài khoản.

2. Truy cập vào đường dẫn của competition [VietAI Foundation Course - CNN Assignment](https://www.kaggle.com/c/vietai-foundation-course-7-cnn-assignment)

3. Nhấn vào nút **Join Competition**.
![alt text](https://storage.googleapis.com/vietai/Screen%20Shot%202019-05-13%20at%2018.48.12.png)

4. Nhấn vào nút **I Understand and Accept**.
![alt text](https://storage.googleapis.com/vietai/Screen%20Shot%202019-05-13%20at%2018.48.52.png)

5. Chọn **Team**.
![alt text](https://storage.googleapis.com/vietai/Screen%20Shot%202019-05-13%20at%2018.49.43.png)

6. Đặt team name theo đúng họ và tên của bạn và bấm **Save team name**.
![alt text](https://storage.googleapis.com/vietai/Screen%20Shot%202019-05-13%20at%2018.50.30.png)

7. Để nộp file CSV vừa tạo, các bạn nhấp vào **Submit Predictions**.
 ![alt text](https://storage.googleapis.com/vietai/Screen%20Shot%202019-05-13%20at%2018.51.39.png)
 
8. Upload file CSV và nộp.
 ![alt text](https://storage.googleapis.com/vietai/Screen%20Shot%202019-05-13%20at%2018.52.19.png)

9. Sau khi nộp, màn hình sẽ hiện ra kết quả, để biết vị trí mình trên leaderboard, các bạn nhấp vào **Jump to your position on the leaderboard**.
 ![alt text](https://storage.googleapis.com/vietai/Screen%20Shot%202019-05-13%20at%2018.55.23.png)

10. Leaderboard sẽ như sau:
 ![alt text](https://storage.googleapis.com/vietai/Screen%20Shot%202019-05-13%20at%2018.55.32.png)

# Thang điểm

- Hoàn tất codes trên Notebook: 7đ
- Kaggle:
  
  + Vượt qua baseline1: 3đ
  + Vượt qua baseline2: +1đ
  + 5 bạn đứng đầu leaderboard của tập private test: +1đ

# Authors: Quoc Pham, Chuong Huynh